# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

##### Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [4]:
from sklearn.model_selection import train_test_split

# Separando os dados em treinamento e teste (75% treino, 25% teste)
df_treino, df_teste = train_test_split(df, test_size=0.25, random_state=42)

# Verificando o tamanho de cada subconjunto
print(f"Tamanho do treino: {len(df_treino)}")
print(f"Tamanho do teste: {len(df_teste)}")

Tamanho do treino: 11250
Tamanho do teste: 3750


##### Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o na base de testes. Qual o melhor modelo?

In [8]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

# Separar X e y
X = df_treino.drop(columns=['renda', 'data_ref', 'id_cliente'], errors='ignore')
y = np.log(df_treino['renda'])  # log da renda

# Base de teste
X_teste = df_teste.drop(columns=['renda', 'data_ref', 'id_cliente'], errors='ignore')
y_teste = np.log(df_teste['renda'])

# Criar dummies para variáveis categóricas
X = pd.get_dummies(X, drop_first=True)
X_teste = pd.get_dummies(X_teste, drop_first=True)

# Garantir mesmas colunas
X_teste = X_teste.reindex(columns=X.columns, fill_value=0)

# Imputar valores ausentes (média para colunas numéricas)
imputer = SimpleImputer(strategy='mean')
X_treino_imp = imputer.fit_transform(X)
X_teste_imp = imputer.transform(X_teste)

# Padronizar as variáveis numéricas
scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino_imp)
X_teste_scaled = scaler.transform(X_teste_imp)

# Testar diferentes valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
resultados = []

for a in alphas:
    ridge = Ridge(alpha=a)
    ridge.fit(X_treino_scaled, y)
    y_pred = ridge.predict(X_teste_scaled)
    r2 = r2_score(y_teste, y_pred)
    resultados.append({'alpha': a, 'R2_teste': r2})

# Exibir resultados
resultados_df = pd.DataFrame(resultados)
print(resultados_df)

# Melhor modelo
melhor_alpha = resultados_df.loc[resultados_df['R2_teste'].idxmax(), 'alpha']
print(f"Melhor modelo: alpha = {melhor_alpha:.3f}")

   alpha  R2_teste
0  0.000  0.350900
1  0.001  0.350900
2  0.005  0.350900
3  0.010  0.350900
4  0.050  0.350899
5  0.100  0.350899
Melhor modelo: alpha = 0.000


##### Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

In [13]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

# Testar os mesmos valores de alpha
alphas = [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1]
resultados_lasso = []

# Rodar o LASSO
for a in alphas:
    lasso = Lasso(alpha=a, max_iter=10000)
    lasso.fit(X_treino_scaled, y)
    y_pred = lasso.predict(X_teste_scaled)
    r2 = r2_score(y_teste, y_pred)
    resultados_lasso.append({'alpha': a, 'R2_teste': r2})

# Resultados do LASSO
resultados_lasso_df = pd.DataFrame(resultados_lasso)
print(resultados_lasso_df)

# Melhor modelo LASSO
melhor_alpha_lasso = resultados_lasso_df.loc[resultados_lasso_df['R2_teste'].idxmax(), 'alpha']
print(f"Melhor modelo LASSO: alpha = {melhor_alpha_lasso:.4f}")

# Comparar Ridge x LASSO
melhor_ridge = resultados_df['R2_teste'].max()
melhor_lasso = resultados_lasso_df['R2_teste'].max()

print("\nComparação final:")
print(f"Ridge  R² = {melhor_ridge:.4f}")
print(f"LASSO  R² = {melhor_lasso:.4f}")

if melhor_lasso > melhor_ridge:
    print("O modelo LASSO apresentou melhor desempenho.")
elif melhor_lasso < melhor_ridge:
    print("O modelo Ridge apresentou melhor desempenho.")
else:
    print("Ambos os modelos tiveram desempenho equivalente.")

    alpha  R2_teste
0  0.0001  0.350784
1  0.0010  0.350649
2  0.0050  0.350396
3  0.0100  0.349075
4  0.0500  0.330409
5  0.1000  0.300432
Melhor modelo LASSO: alpha = 0.0001

Comparação final:
Ridge  R² = 0.3509
LASSO  R² = 0.3508
O modelo Ridge apresentou melhor desempenho.


##### Rode um modelo stepwise. Avalie o na base de testes. Qual o melhor resultado?

In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Garantir que não há valores ausentes na variável usada
df = df.dropna(subset=['tempo_emprego', 'renda'])

# Criar dummies e converter tudo para float
df_dummies = pd.get_dummies(df.drop(columns=['data_ref', 'id_cliente']),
                            drop_first=True).astype(float)

# Separar variáveis dependente e independentes
X = df_dummies.drop(columns=['renda'])
y = np.log(df_dummies['renda'])

# Dividir treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# Função stepwise com tratamento para tipos numéricos
def stepwise_selection(X, y, initial_list=[], threshold_in=0.05, threshold_out=0.10, verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_col in excluded:
            try:
                model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_col]]))).fit()
                new_pval[new_col] = model.pvalues[new_col]
            except Exception:
                new_pval[new_col] = np.nan
        best_pval = new_pval.min()
        if best_pval is not None and best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f"Adicionado: {best_feature} com p={best_pval:.6f}")

        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]  # exclui intercepto
        worst_pval = pvalues.max()
        if worst_pval is not None and worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print(f"Removido: {worst_feature} com p={worst_pval:.6f}")

        if not changed:
            break

    return included

# Executar seleção stepwise
variaveis_final = stepwise_selection(X_treino, y_treino)

# Ajustar modelo final
modelo_stepwise = sm.OLS(y_treino, sm.add_constant(X_treino[variaveis_final])).fit()

# Avaliar desempenho no teste
y_pred = modelo_stepwise.predict(sm.add_constant(X_teste[variaveis_final]))
r2_teste = r2_score(y_teste, y_pred)

print("\nVariáveis selecionadas:")
print(variaveis_final)
print(f"\nR² na base de teste: {r2_teste:.4f}")
print("\nResumo do modelo:")
print(modelo_stepwise.summary())

Adicionado: tempo_emprego com p=0.000000
Adicionado: sexo_M com p=0.000000
Adicionado: tipo_renda_Empresário com p=0.000000
Adicionado: educacao_Superior completo com p=0.000000
Adicionado: idade com p=0.000000
Adicionado: posse_de_imovel com p=0.000000
Adicionado: qtd_filhos com p=0.002739
Adicionado: tipo_renda_Servidor público com p=0.012171
Adicionado: estado_civil_Viúvo com p=0.015638

Variáveis selecionadas:
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'educacao_Superior completo', 'idade', 'posse_de_imovel', 'qtd_filhos', 'tipo_renda_Servidor público', 'estado_civil_Viúvo']

R² na base de teste: 0.3638

Resumo do modelo:
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.352
Method:                 Least Squares   F-statistic:                     564.1
Date:                Thu, 30 Oct 2025   

##### Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

Ao comparar os parâmetros entre os modelos Ridge, LASSO e Stepwise, observa-se que as magnitudes e os sinais dos coeficientes principais permanecem consistentes, indicando estabilidade das relações entre as variáveis explicativas e a renda. O modelo Stepwise apresentou um R² ligeiramente superior (0.364) e manteve apenas variáveis estatisticamente significantes (p < 0.05), demonstrando maior parcimônia sem perda de desempenho. Assim, o modelo Stepwise é o mais adequado entre os testados, por combinar boa capacidade preditiva com simplicidade e coerência estatística.

##### Partindo dos modelos que você ajustou, tente melhorar o na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

Abaixo, código aplica técnicas de engenharia de variáveis para melhorar o desempenho do modelo de regressão. A ideia é capturar relações não lineares e interações entre variáveis que o modelo linear simples não consegue representar. Para isso, foram criadas transformações como quadrados, produtos e razões entre variáveis relevantes (por exemplo, idade², idade * tempo_emprego e renda_per_capita). Em seguida, as variáveis categóricas foram convertidas em dummies, os dados padronizados e o modelo Ridge foi reajustado com o mesmo parâmetro ótimo de regularização. Esse processo busca aumentar o R² na base de teste, mantendo um equilíbrio entre precisão e simplicidade do modelo

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# Cópia da base já tratada
df_mod = df.copy()

# Criação de novas variáveis (exemplos)
df_mod['idade2'] = df_mod['idade'] ** 2
df_mod['tempo_emprego2'] = df_mod['tempo_emprego'] ** 2
df_mod['idade_tempo'] = df_mod['idade'] * df_mod['tempo_emprego']
df_mod['filhos_por_pessoa'] = df_mod['qtd_filhos'] / (df_mod['qt_pessoas_residencia'] + 1)
df_mod['renda_per_capita'] = df_mod['renda'] / (df_mod['qt_pessoas_residencia'] + 1)

# Remover valores ausentes e gerar dummies
df_mod = df_mod.dropna(subset=['tempo_emprego'])
df_dummies = pd.get_dummies(df_mod.drop(columns=['data_ref', 'id_cliente']), drop_first=True).astype(float)

# Divisão treino/teste
X = df_dummies.drop(columns=['renda'])
y = np.log(df_dummies['renda'])

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

# Padronizar
scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

# Reajustar modelo (ex: Ridge com melhor alpha anterior)
modelo = Ridge(alpha=0.001)
modelo.fit(X_treino_scaled, y_treino)

# Avaliar
y_pred = modelo.predict(X_teste_scaled)
r2_teste = r2_score(y_teste, y_pred)
print(f"Novo R² na base de teste: {r2_teste:.4f}")

Novo R² na base de teste: 0.6374


O script abaixo compara o modelo Ridge original (com variáveis simples) com uma versão melhorada que inclui transformações não lineares e interações (como idade² e idade * tempo_emprego). Ambas as versões passam por padronização e uso de Ridge com o mesmo alpha para manter a comparação justa.
Por fim, o código exibe os dois valores de R² na base de teste, permitindo avaliar se as novas variáveis realmente aumentaram o poder preditivo do modelo.
Esse tipo de abordagem é essencial em modelagem estatística, pois busca melhorar a performance sem comprometer a interpretabilidade

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# ==========================
# Base original
# ==========================
df_original = df.copy()
df_original = df_original.dropna(subset=['tempo_emprego'])

# Criar dummies
df_original_dummies = pd.get_dummies(df_original.drop(columns=['data_ref', 'id_cliente']), drop_first=True).astype(float)

# Separar treino e teste
X = df_original_dummies.drop(columns=['renda'])
y = np.log(df_original_dummies['renda'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Padronizar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ajustar modelo Ridge
ridge_original = Ridge(alpha=0.001)
ridge_original.fit(X_train_scaled, y_train)
r2_original = r2_score(y_test, ridge_original.predict(X_test_scaled))

# ==========================
# Base com transformações
# ==========================
df_mod = df.copy()
df_mod['idade2'] = df_mod['idade'] ** 2
df_mod['tempo_emprego2'] = df_mod['tempo_emprego'] ** 2
df_mod['idade_tempo'] = df_mod['idade'] * df_mod['tempo_emprego']
df_mod['filhos_por_pessoa'] = df_mod['qtd_filhos'] / (df_mod['qt_pessoas_residencia'] + 1)
df_mod = df_mod.dropna(subset=['tempo_emprego'])
df_mod_dummies = pd.get_dummies(df_mod.drop(columns=['data_ref', 'id_cliente']), drop_first=True).astype(float)

X2 = df_mod_dummies.drop(columns=['renda'])
y2 = np.log(df_mod_dummies['renda'])
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.25, random_state=42)

scaler2 = StandardScaler()
X2_train_scaled = scaler2.fit_transform(X2_train)
X2_test_scaled = scaler2.transform(X2_test)

ridge_mod = Ridge(alpha=0.001)
ridge_mod.fit(X2_train_scaled, y2_train)
r2_mod = r2_score(y2_test, ridge_mod.predict(X2_test_scaled))

# ==========================
# Comparação dos resultados
# ==========================
print(f"R² base original: {r2_original:.4f}")
print(f"R² base transformada: {r2_mod:.4f}")
if r2_mod > r2_original:
    print("O modelo com transformações apresentou melhor desempenho.")
else:
    print("O modelo original teve desempenho semelhante ou superior.")

R² base original: 0.3647
R² base transformada: 0.3669
O modelo com transformações apresentou melhor desempenho.


##### Ajuste uma árvore de regressão e veja se consegue um melhor com ela

In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Cópia da base já tratada
df_tree = df.copy()

# Remover colunas irrelevantes e valores ausentes
df_tree = df_tree.drop(columns=['data_ref', 'id_cliente'])
df_tree = df_tree.dropna(subset=['tempo_emprego'])

# Criar dummies para variáveis categóricas
df_tree_dummies = pd.get_dummies(df_tree, drop_first=True).astype(float)

# Separar variáveis explicativas e resposta
X = df_tree_dummies.drop(columns=['renda'])
y = np.log(df_tree_dummies['renda'])

# Dividir em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

# Ajustar a árvore de regressão
tree = DecisionTreeRegressor(max_depth=6, min_samples_leaf=50, random_state=42)
tree.fit(X_treino, y_treino)

# Avaliar desempenho
y_pred = tree.predict(X_teste)
r2_tree = r2_score(y_teste, y_pred)
mse_tree = mean_squared_error(y_teste, y_pred)

print(f"R² (base de teste): {r2_tree:.4f}")
print(f"MSE: {mse_tree:.4f}")

R² (base de teste): 0.3734
MSE: 0.5102


O modelo de árvore de regressão apresentou o melhor desempenho entre todos os testados, com R² de 0.3734 e MSE de 0.5102 na base de teste. Isso indica que ele conseguiu capturar relações não lineares e interações entre as variáveis que os modelos lineares (Ridge, LASSO e Stepwise) não conseguiram representar adequadamente. Assim, a árvore de regressão se mostra uma alternativa mais eficaz para prever a renda, oferecendo maior capacidade preditiva sem a necessidade de suposições estritas de linearidade.